**Goals for this Notebook:**

Having conducted an initial analysis, I have gained a comprehensive understanding of the dataset, including valuable insights into the main customer segments and the overall data structure.

In this notebook, I will focus on analyzing the results of our A/B testing. The process is structured as follows:

**Chapter 00: Files preparation**
- Ensure all necessary files are correctly prepared and loaded for the analysis.
  
**Chapter 01: Defining the Metrics Base**
  - Define the criteria for a valid session, which involves setting specific rules to accurately capture user visits to the website.
  - Define the rest of the metrics for the data: steps back, total steps etc.

# 00. Files preparation

## Importing libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches # used for the legend at step duration
import seaborn as sns
import numpy as np
import datetime
import scipy.stats as st
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
from scipy.stats import norm
import scipy.stats as stats

from functions import cleaning 

from functools import reduce

In [2]:
# Disabling a specific type of warning from Jupyter, that I know if mistaken for something else
pd.options.mode.chained_assignment = None 

## Loading dataset

Datasets provided by Ironhack.

In [3]:
filepath1 = '../Data/df_final_demo.txt' # Client Profiles
filepath2 = '../Data/df_final_web_data_pt_1.txt' # Digital Footprints, part 1
filepath3 = '../Data/df_final_web_data_pt_2.txt' # Digital Footprints, part 2
filepath4 = '../Data/df_final_experiment_clients.txt' # Experiment Roster 

# Loading datasets
df_clients = pd.read_csv(filepath1)
df2 = pd.read_csv(filepath2)
df3 = pd.read_csv(filepath3)
df_roster = pd.read_csv(filepath4)

# Merging part 1 and 2 of Digital Footprints
df_trace = pd.concat([df2, df3], axis=0, ignore_index=True)

## Data cleaning

In [4]:
# Calling the cleaning function I created at the functions.py file
df_clients, df_trace, df_roster, _ = cleaning(df_clients, df_trace, df_roster)

# 01. Defining the Metrics Base

After inspecting the data, I noticed some inconsistencies and possible errors in the **visit_id**. This is important because for the A/B testing analysis, I need to accurately determine how many clients complete the process and compare the two groups (test and control).

Specifically, I observed that for the same client, the visit_id sometimes changed even when there were only a few seconds between steps (likely due to page refreshes). Conversely, the same visit_id sometimes spanned steps that were days apart.

In a real-life scenario, I would consult the IT team to understand the criteria for assigning visit_id and decide whether it can be trusted. Additionally, I would align this with the business goals, considering the user session duration that the business aims to achieve.

**Since this project uses a public dataset, I will define my own rules for what constitutes a valid session.** To do this, I will:
1. **Calculate Time Between Steps:** Calculate the time between successful steps (steps that move forward, not backward).
2. **Identify Outliers:** Use statistical methods to identify outliers in the time between steps.
3. **Prepare the Files:** Add necessary columns and prepare the data to facilitate the definition of valid sessions.
4. **Define Valid Sessions:** Based on the findings, establish rules to define what constitutes a valid session.
5. **Define Session Metrics:** Preparing the data to measure more metrics like steps back, total step count etc.

## Calculate time between moving forward steps

In [5]:
# will use all the data for that (both from test and control group)
# making a copy to not change the original dataframe
df_step_time = df_trace.copy()

# Changing step names, sorting dataframe and resseting index, to help with the analysis below
df_step_time['process_step'] = df_step_time['process_step'].map({'start':1,'step_1':2,'step_2':3,'step_3':4,'confirm':5})
df_step_time.sort_values(by=['visit_id','date_time'], ascending=False, inplace=True)
df_step_time = df_step_time.reset_index(drop=True)

# Adding new columns with previous visit_id, previous process step and previous time, to help with the analysis below
df_step_time['prev_visit_id'] = df_step_time['visit_id'].shift(-1, fill_value=0)
df_step_time['prev_process_step'] = df_step_time['process_step'].shift(-1, fill_value=0)
df_step_time['prev_date_time'] = df_step_time['date_time'].shift(-1, fill_value='1970-01-01 00:00:00')


# Converting to datetime
df_step_time['date_time'] = pd.to_datetime(df_step_time['date_time'],format='%Y-%m-%d %H:%M:%S')
df_step_time['prev_date_time'] = pd.to_datetime(df_step_time['prev_date_time'],format='%Y-%m-%d %H:%M:%S')


# Adding a new layer, to check how the current step and the previous step compare
df_step_time['subtracting_step'] = df_step_time['process_step'] - df_step_time['prev_process_step']

# Adding a new layer, to find the time between successful steps, and filter out the ones with no useful info
df_step_time['subtracting_date_time'] = df_step_time['date_time'] - df_step_time['prev_date_time']


# Filtering the sample, to the interactions that moved one step forward == 1
# and excluding the ones that had substraction with the placeholder date '1970-01-01 00:00:00' < 10000 days
filtered_df_step_time = df_step_time[(df_step_time['subtracting_step'] == 1) & (
    df_step_time['subtracting_date_time'] < '10000 days 00:00:00') & (
    df_step_time['visit_id'] == df_step_time['prev_visit_id'])]

In [6]:
filtered_df_step_time['subtracting_date_time'].describe()

count                       442623
mean     0 days 00:01:00.451167246
std      0 days 00:01:41.177206532
min                0 days 00:00:00
25%                0 days 00:00:12
50%                0 days 00:00:31
75%                0 days 00:01:09
max                0 days 01:00:36
Name: subtracting_date_time, dtype: object

In [7]:
filtered_df_step_time['subtracting_date_time'].quantile(0.95) #  3 minutes at the 95% percentile

Timedelta('0 days 00:03:25')

### Defining outliers with statistical methods

In [8]:
# defining outliers with the IQR method
IQR = filtered_df_step_time['subtracting_date_time'].quantile(0.75) - filtered_df_step_time['subtracting_date_time'].quantile(0.25)
upper_limit_for_outliers = filtered_df_step_time['subtracting_date_time'].quantile(0.75) + IQR
upper_limit_for_outliers

Timedelta('0 days 00:02:06')

In [9]:
# defining outliers with standard deviation method
filtered_df_step_time['subtracting_date_time'].mean() + filtered_df_step_time['subtracting_date_time'].std()*3

Timedelta('0 days 00:06:03.982786842')

By this sample analysis, it appears that the average duration for a step forward in the online interaction is approximately 2 minutes, with the majority of the observations(95% percentile) within a range of around 3,5 minutes. **To identify outliers, I utilized statistical methods, opting for the Standard Deviation method (multpiplied by 3)**, which indicates that **anything exceeding 6 minutes and 4 seconds should be considered an outlier**.  

I will incorporate the identified duration as a threshold to distinguish between sessions in digital footstep analysis. Considering typical intervals between steps of less than the threshold will help determine continuity within sessions versus potential new sessions.

## Data preparation, Splitting dataset for test and control group

In [10]:
# First separating the datasets for the test and control group
df_roster_test = df_roster[df_roster.variation == 'Test']
df_roster_control = df_roster[df_roster.variation == 'Control']

df_test = df_trace[df_trace.client_id.isin(df_roster_test.client_id.unique().tolist())]
df_control = df_trace[df_trace.client_id.isin(df_roster_control.client_id.unique().tolist())]

In [11]:
def df_prep(df):
    ''' Preparing the data for analysis.
    I created the function, cause the same preparation will be done for two datasets (for test and control group).

    Receives dataframe as parameter and returns updated dataframe.'''

    # Rename column for brevity
    df = df.rename(columns={'process_step': 'step'})

    # Replacing the step names with numbers for analysis purposes (will do subtraction later)
    df['step']  = df['step'].map({'start':1,'step_1':2,'step_2':3,'step_3':4,'confirm':5})
    
    # Sorting dataframe
    df.sort_values(by=['client_id','date_time'], ascending=False, inplace=True)

    # Dropping the columns that are not needed
    df.drop(columns=['visitor_id','visit_id'], inplace=True)
   
    # Resetting index
    df = df.reset_index(drop=True)

    
    # New columns for analysis
    # Adding new columns that I will use for comparison: previous  and next client_id
    df['prev_id'] = df['client_id'].shift(-1, fill_value=0)
    df['next_id'] = df['client_id'].shift(1, fill_value=0)

    # Adding new columns that I will use for comparison: previous and next step
    df['prev_step'] = df['step'].shift(-1, fill_value=0)
    df['next_step'] = df['step'].shift(1, fill_value=0)

    # Adding new columns that I will use for comparison: previous datetime, next datetime
    df['prev_time'] = df['date_time'].shift(-1, fill_value='1970-01-01 00:00:00')
    df['prev_time'] = pd.to_datetime(df['prev_time'],format='%Y-%m-%d %H:%M:%S')

    df['next_time'] = df['date_time'].shift(1, fill_value='1970-01-01 00:00:00')
    df['next_time'] = pd.to_datetime(df['next_time'],format='%Y-%m-%d %H:%M:%S')

    # Adding new columns that I will use for comparison: time difference between current and last step
    df['time_diff_prev'] = df['date_time'] - df['prev_time']

    # Adding new columns that I will use for comparison: step duration (difference of following datime with current one)
    df['time_diff_next'] = df['next_time'] - df['date_time']

    # Adding a new layer, to count the steps back, but making sure that I don't compare different clients
    df['subtract_step'] = np.where(df['client_id'] == df['prev_id'],(df['step'] - df['prev_step']),0)

    
    return df

In [12]:
df_test = df_prep(df_test)
df_control = df_prep(df_control)

## Define valid sessions

**Conditions for Defining Valid Sessions:**

The condition outlined below aim to accurately identify the start of a new session within a sequence of actions:
- **current action is 'step' 1**

   **AND**
- **the previous step is more than 6 minutes and 4 seconds ago OR the previous entry is from a different client**



In [13]:
# here I will add to my dataframes for the test and control group a column that will signify a new session
# creating a definition cause I will do the same for both dataframes (control and test)

def new_session(df):
    ''' Function that signifies which point at the digital fottprints are new sessions, based on the conditions:
    - current action is 'step' 1
    AND
    - the previous step is more than 6 min and 4 s ago OR the previous entry is from a different client

    Receives the dataframe as parameter.
    Returns dataframe with an extra column 'new_session' with True or False.'''

    df['new_session'] = False

    # Threshold of 6 min and 4 seconds as calculated above
    threshold = datetime.timedelta(seconds=364)

    # setting the conditions
    conditions = np.array(
    (df['step'] == 1) & 
    ((df['time_diff_prev'] > threshold) |
    (df['client_id'] != df['prev_id']))
    )

    df.loc[conditions, 'new_session'] = True
    
    # Sorting everything to ascending order, to help with following steps
    df.sort_values(by=['client_id','date_time'], ascending=True, inplace=True)

    return df

In [14]:
df_test = new_session(df_test)
df_control = new_session(df_control)

In [15]:
# Numbering seperate sessions
count = 0  # Initialize count outside the function

def session_id(new_session):
    ''' Function to number the seperate sessions'''
    
    global count  # Access the count variable from outside the function
    
    if new_session == True:
        count += 1
        return count       
    else:
        return count


# Calling session_id that is defined in functions.py
df_test['session_id'] = df_test['new_session'].apply(session_id)


# Doing the same for the 'control' dataframe
count = 0  # Resetting count outside the function

df_control['session_id'] = df_control['new_session'].apply(session_id)

## Session metrics

Conditions:

**Valid Session**:
A valid session is characterized by consecutive steps occurring within a duration of no more than 6 minutes and 4 seconds.

**Invalid Session**:
An invalid session is identified when the time interval between consecutive steps exceeds 6 minutes and 4 seconds.

**Successful Session**:
A successful session is a valid session that successfully reaches the 'confirm' stage after traversing all required steps ('start', 'step1', 'step2', 'step3'). In cases of multiple 'confirm' steps, the last one is regarded as the official confirm. Metrics for successful sessions include quantifying the total number of steps taken, the total number of steps taken backward, and calculating the duration from the session start to the final valid confirm step.

**Unsuccessful Session**:
Unsuccessful session is one that fails to reach the 'confirm' stage completely or reaches 'confirm' without passing through the necessary steps. These sessions may be flagged for further investigation to understand the reasons behind the incomplete or irregular user behavior.

In [16]:
# Variable to count steps until 'confirm'
steps_count = 1
# Variable to count how many steps back there were per session
steps_back = 0
# Variable to count how many valid 'confirms' have been per session
confirm_count = 0
# Variable to check if there are more than 1 'confirm' in the same session
confirmed = False
# Variable to check if the session is valid
valid = True
# Variables to see if per session steps 1,2,3 were completed before 'confirm'
check_1 = False
check_2 = False
check_3 = False
# Variable to measure how long a session that reached 'confirm' lasted
session_start = '1970-01-01 00:00:00'


def session_info(df):
    # To access the variable from outside the function
    global steps_count 
    global steps_back
    global confirm_count
    global confirmed
    global valid
    global check_1
    global check_2
    global check_3
    global session_start

    threshold = datetime.timedelta(seconds=364)
    
    if df['new_session'] == True:
        # Resetting the counts for the new sessions
        steps_count = 1
        steps_back = 0
        confirm_count = 0
        confirmed = False
        valid = True
        session_start = df['date_time']
        session_start = pd.to_datetime(session_start,format='%Y-%m-%d %H:%M:%S')
        check_1 = False
        check_2 = False
        checl_3 = False        
        return [confirm_count, steps_count, steps_back, df['time_diff_next'], pd.NaT, 'valid']

    elif df['time_diff_prev'] < datetime.timedelta(seconds=0):
        # this refers to the few cases, that have the same session id, but different client_id, etc
        # and they are in the same session_id because the new session needs 'start' to be defined.
        # if the next client just started from any other valid, it is invalid, but currently it is 
        # noted as the same session as the one above it.
        valid = False
        return [confirm_count, steps_count, steps_back, pd.NaT, pd.NaT, 'invalid']
    
    elif df['time_diff_prev'] < threshold and valid:
        
        if df['step'] in (1,2,3,4):
            if df['step'] == 2:
                check_1 = True
            elif df['step'] == 3:
                check_2 = True
            else:
                check_3 = True
                
            # Counting the steps back
            if df['subtract_step'] < 0:
                steps_back += abs(df['subtract_step'])            
            
            # Counting the total steps
            steps_count += 1

            return [confirm_count, steps_count, steps_back, df['time_diff_next'], pd.NaT, 'valid']
 
        # At the first valid 'confirm' I am returning the total number of steps
        elif df['step'] == 5 and not confirmed and check_1 and check_2 and check_3:
            confirm_count = 1
            confirmed = True
            steps_count += 1
            session_dur = df['date_time'] - session_start
            return [confirm_count, steps_count, steps_back, pd.NaT, session_dur,'valid']
        
        # When more than one 'confirm' happen within the valid timeframe, I need to return the updated info, 
        # and add a step count for the first 'confirm'
        elif df['step'] == 5 and confirmed and check_1 and check_2 and check_3:
            confirm_count += 1
            session_dur = df['date_time'] - session_start
            steps_count += 1
            return [confirm_count, steps_count, steps_back, pd.NaT, session_dur,'valid']
    
        # Checking the case where'confirm' is reached without passing from all the necessary steps
        elif df['step'] == 5 and not confirmed:
            # I am considering it a valid session but not a successful one
            # I will count it as an extra step, but that can be defined better later
            steps_count += 1
            return [confirm_count, steps_count, steps_back, df['time_diff_next'], pd.NaT, 'valid']
    
    elif df['time_diff_prev'] > threshold and valid:
        valid = False
        return [confirm_count, steps_count, steps_back, pd.NaT, pd.NaT, 'invalid']

    else:
        # this is the case where valid is already False
        return [confirm_count, steps_count, steps_back, pd.NaT, pd.NaT, 'invalid']

In [17]:
# Applying the above function
df_test['session_info'] = df_test.apply(session_info, axis=1)
df_control['session_info'] = df_control.apply(session_info, axis=1)


# Dropping columns that will not be needed further, saving it to a new dataframe with results
df_test_rslt = df_test.drop(columns=(['time_diff_next','time_diff_prev','new_session',
                       'prev_id','next_id','prev_step','next_step','prev_time','next_time']))
df_control_rslt = df_control.drop(columns=(['time_diff_next','time_diff_prev','new_session',
                       'prev_id','next_id','prev_step','next_step','prev_time','next_time']))


# Splitting the session info in more columns
df_test_rslt[['confirm_cnt', 'steps_cnt', 'steps_back_cnt', 'step_dur',
              'sess_dur_to_confirm','validity']] = df_test_rslt['session_info'].apply(pd.Series)
df_control_rslt[['confirm_cnt', 'steps_cnt', 'steps_back_cnt', 'step_dur',
              'sess_dur_to_confirm','validity']] = df_control_rslt['session_info'].apply(pd.Series)


# Dropping the session_info column
df_test_rslt = df_test_rslt.drop(columns=('session_info'))
df_control_rslt = df_control_rslt.drop(columns=('session_info'))


# Correcting invalid step_dur, if more than 6min and 2 sec, they are result of subtracting with the next session's time
conditions = (df_test_rslt['step_dur'] > datetime.timedelta(seconds=363)) | (df_test_rslt['step_dur'] < datetime.timedelta(seconds=0))
df_test_rslt.loc[conditions,'step_dur'] = pd.NaT

conditions = (df_control_rslt['step_dur'] > datetime.timedelta(seconds=363)) | (df_control_rslt['step_dur'] < datetime.timedelta(seconds=0))
df_control_rslt.loc[conditions,'step_dur'] = pd.NaT

# Transforming the duration to seconds
df_test_rslt['step_dur'] = df_test_rslt['step_dur'].dt.total_seconds().apply(lambda x: '{:.0f}'.format(x))
df_control_rslt['step_dur'] = df_control_rslt['step_dur'].dt.total_seconds().apply(lambda x: '{:.0f}'.format(x))

In [18]:
df_test_rslt.head()

,client_id,step,date_time,subtract_step,session_id,confirm_cnt,steps_cnt,steps_back_cnt,step_dur,sess_dur_to_confirm,validity
176698,1001101,1,2017-04-05 16:55:20,0,1,0,1,0,13,NaT,valid
176697,1001101,2,2017-04-05 16:55:33,1,1,0,2,0,29,NaT,valid
176696,1001101,3,2017-04-05 16:56:02,1,1,0,3,0,32,NaT,valid
176695,1001101,2,2017-04-05 16:56:34,-1,1,0,4,1,26,NaT,valid
176694,1001101,3,2017-04-05 16:57:00,1,1,0,5,1,20,NaT,valid


In [19]:
df_control_rslt.head()

,client_id,step,date_time,subtract_step,session_id,confirm_cnt,steps_cnt,steps_back_cnt,step_dur,sess_dur_to_confirm,validity
140535,1000062,1,2017-04-06 15:38:49,0,1,0,1,0,182,NaT,valid
140534,1000062,2,2017-04-06 15:41:51,1,1,0,2,0,10,NaT,valid
140533,1000062,3,2017-04-06 15:42:01,1,1,0,3,0,49,NaT,valid
140532,1000062,4,2017-04-06 15:42:50,1,1,0,4,0,nan,NaT,valid
140531,1000062,5,2017-04-06 15:54:49,1,1,0,4,0,nan,NaT,invalid


# Saving files

In [20]:
# df_test_rslt.to_csv('../Data/df_test_rslt.txt')
# df_control_rslt.to_csv('../Data/df_control_rslt.txt')